# Animation Demo

Let's animate a planet orbiting a star!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter

%config InlineBackend.figure_format='retina' # makes animation display better
# ^ COMMENT OUT IF ON WINDOWS

%matplotlib osx 
# ^ UNCOMMENT THIS LINE IF USING MAC

# %matplotlib qt 
# ^ UNCOMMENT THIS LINE IS USING WINDOWS

Set up initial conditions:

In [ ]:
# Define masses
mp = 3 * 10**(-5)
ms = 1.0

# Define initial position vectors 
r1 = np.array([1, 0])
r2 = np.array([0, 0]) # at orgin

# Define initial velocities
v1 = np.array([0, 0.75])
v2 = np.array([0, 0]) # stationary

Newton's Law of Universal Gravitation governs our planet's motion:

\begin{equation}
 F = G \frac{m_1 m_2}{r^2}
\end{equation}

We define a function that calculates velocity and acceleration at some time step t:

In [ ]:
def OrbitEquation(w, t, m1, m2): # w is an array containing positions and velocities
    r1 = w[:2]
    v1 = w[2:4]
    
    r12 = np.linalg.norm(r1)
    
    dv1bydt = m2*(-r1)/r12**3  # derivative of velocity

    dr1bydt = v1 # derivative of position 
    
    r_derivs = dr1bydt
    v_derivs = dv1bydt
    derivs = np.concatenate((r_derivs, v_derivs)) # joining the two arrays
    
    return derivs

Solve the ordinary differential equation (OrbitEquation) using scipy.integrate.odeint:

In [ ]:
# Package initial parameters into one array (just easier to work with this way)
init_params = np.array([r1, v1])
init_params = init_params.flatten()
time_span = np.linspace(0, 5, 500)  # run for t=5 (500 points)

# Run the ODE solver
sol = scipy.integrate.odeint(OrbitEquation, init_params, time_span, args=(mp,ms))

Save the solutions into a format that's easier to work with: 

In [ ]:
r1_sol = sol[:,:2]

Initialize the writer:

In [ ]:
# Initilize writer 
metadata = dict(title='My first animation', artist='Matplotlib')
writer = FFMpegWriter(fps=50, metadata=metadata, bitrate=200000) # change fps for different frame rates
fig = plt.figure(dpi=200)

Running the writer loop:

(this should open up a separate window and it will most likely take a while to run!)

In [ ]:
# SAVE AS MP4 (will be saved in whatever directory you are working in)
fig, ax = plt.subplots()

with writer.saving(fig, "orbit.mp4", dpi=200):
    for i in range(len(time_span)):

        ax.clear()

        ax.plot(r1_sol[:i,0],r1_sol[:i,1],color="blue", alpha=0.5)
        ax.scatter(r1_sol[i,0],r1_sol[i,1],color="blue",marker="o",s=20, zorder=5) # planet
        
        ax.scatter(0, 0, color="orange",marker="*",s=50, zorder=5) # star
        
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

****

Plotting a dynamic tail!

In [ ]:
# SAVE AS MP4 -- tailed orbit
fig, ax = plt.subplots()

tail_length = 50  # set tail length

with writer.saving(fig, "orbit_tail_.mp4", dpi=200):
    for i in range(len(time_span)):

        ax.clear()
        
        if i < tail_length:
            ax.plot(r1_sol[:i,0],r1_sol[:i,1],color="blue", alpha=0.5)
        else:
            ax.plot(r1_sol[i-tail_length:i,0],r1_sol[i-tail_length:i,1],color="blue", alpha=0.5)
            
        ax.scatter(r1_sol[i,0],r1_sol[i,1],color="blue",marker="o",s=20, zorder=5) # planet
        
        ax.scatter(0, 0, color="orange",marker="*",s=50, zorder=5) # star
        
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()